In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from stemming.porter2 import stem

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tag.perceptron import PerceptronTagger

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
# tagger = PerceptronTagger()
# tagset = None
# def remove_grammar(review):
#     sentences = nltk.sent_tokenize(review)
#     sentences = [nltk.word_tokenize(sent) for sent in sentences]
#     print('sentences',sentences)
#     result_review = []
#     for sentence in sentences:
#         tagged_review = nltk.tag._pos_tag(sentence, tagset, tagger)
#         cp = nltk.RegexpParser(grammar)
#         result = cp.parse(tagged_review)
#         result_review.append(traverseTree(result))
#     return ''.join([word for word in result_review])

# def clean(text):
#     text = text.lower()
#     string = ''
    
#     text=re.sub(r"[-()\"#!@$%^&*{}?.,:]"," ",text)
#     text=re.sub(r"\s+"," ",text)
#     text=re.sub('[^A-Za-z0-9]+',' ', text)
    
#     text = remove_grammar(text)
    
#     for word in text.split():
#         if word not in stopwords:
#             string += WordNetLemmatizer().lemmatize(word, pos='v')+ ' '
    
#     return string



In [15]:
hi = 'hi im trying my best'

print(clean(hi))

sentences [['hi', 'im', 'trying', 'my', 'best']]


NotImplementedError: Currently, NLTK pos_tag only supports English and Russian (i.e. lang='eng' or lang='rus')

In [16]:
tagger = PerceptronTagger()
tagset = None
stop = nltk.corpus.stopwords
wordnet_lemmatizer = WordNetLemmatizer()

grammar = '''REMOVE: {<PRP><VBP>?<VBG><TO>?}
                         {<PRP><MD><VB><TO>}
                         {<VBZ><DT><JJ>}
                         {<MD><DT><NN>}
                         {<NNP><PRP><VBP>}
                         {<MD><PRP>}
                         {<NNP><PRP><VBP>}
                         {<WDT><MD>}
                         {<PRP><VBP><VBG><VB><DT>}
                         {<VBZ><DT><JJ>}
                         {<VBZ><EX><NN><PRP><VBP><TO><VB>}
                         {<DT><VBZ>}
                         {<PRP><VBP><VBG><TO>}
                         {<MD><VB><TO><VB>}
                         {<VBZ><EX><DT>}
                         {<VB><TO>}
                         {<VBZ>}
                         {<DT>}
                         {<EX>}
                         {<PRP><VBP>}
                         {<CD>}
                         {<PRP\$>}
                         {<PRP>}
                         {<TO>}
                         {<IN>}
                         {<VBP>}
                         {<CC>}
              '''

def stem_doc(x):
    red_text = [stem(word.strip()) for word in x.split(" ") if word.strip()!='']
    return ' '.join(red_text)

def lem(x):
    try:
        return wordnet_lemmatizer.lemmatize(x,pos='v')
    except:
        return x
        
def remove_url(x):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)

def cleanse_text(text):
    if text:
        text = remove_url(text)
        addl_txt = addl_clean_words(text)
        red_text = clean_words(addl_txt)
        
        no_gram = red_text
        try:
            no_gram = remove_grammar(red_text)
        except:
            no_gram = red_text
    
        #clean = ' '.join([i for i in no_gram.split() if i not in stop])
        if no_gram:
            clean = ' '.join([i for i in no_gram.split()])
            red_text = [lem(word) for word in clean.split(" ")]
            red_text = [stem(word) for word in clean.split(" ")]
            return clean_words(' '.join(red_text))
        else:
            return no_gram
    else:
        return text

def cleanse_text_guided(text):
    if text:
        text = remove_url(text)
        addl_txt = addl_clean_words(text)
        red_text = clean_words_guided(addl_txt)
        
        no_gram = red_text
        try:
            no_gram = remove_grammar(red_text)
        except:
            no_gram = red_text
    
        #clean = ' '.join([i for i in no_gram.split() if i not in stop])
        if no_gram:
            clean = ' '.join([i for i in no_gram.split()])
            red_text = [lem(word) for word in clean.split(" ")]
            red_text = [stem(word) for word in clean.split(" ")]
            return clean_words(' '.join(red_text))
        else:
            return no_gram
    else:
        return text

        
def addl_clean_words(words):
    # any additional data pre-processing
    words = words.replace('can\'t','cannot')
    words = words.replace('won\'t','would not')
    words = words.replace('doesn\'t','does not')
    return words
    
def clean_words(words):
    if words:
        words = remove_email(words)
        words = words.replace('\t',' ')
        words = words.replace(',',' ')
        words = words.replace(':',' ')
        words = words.replace(';',' ')
        words = words.replace('=',' ')
        #words = words.replace('\x92','') # apostrophe encoding
        words = words.replace('\x08','\\b') # \b is being treated as backspace
        #words = ''.join([i for i in words if not i.isdigit()])
        words = words.replace('_',' ')
        words = words.replace('(',' ')
        words = words.replace(')',' ')
        words = words.replace('+',' ')
        words = words.replace('-',' ')
        words = words.replace('`',' ')
        words = words.replace('\'',' ')
        words = words.replace('.',' ')
        words = words.replace('#',' ')
        words = words.replace('/',' ')
        words = words.replace('_',' ')
        words = words.replace('"',' ')
        return words.strip()
    return words

def clean_words_guided(words):
    if words:
        words = remove_email(words)
        words = words.replace('\t',' ')
        words = words.replace(',',' ')
        words = words.replace(':',' ')
        words = words.replace(';',' ')
        words = words.replace('=',' ')
        #words = words.replace('\x92','') # apostrophe encoding
        words = words.replace('\x08','\\b') # \b is being treated as backspace
        #words = ''.join([i for i in words if not i.isdigit()])
        words = words.replace('_',' ')
        words = words.replace('(',' ')
        words = words.replace(')',' ')
        words = words.replace('+',' ')
        words = words.replace('-',' ')
        words = words.replace('`',' ')
        words = words.replace('\'',' ')
        words = words.replace('.',' ')
        words = words.replace('#',' ')
        words = words.replace('/',' ')
        words = words.replace('_',' ')
        words = words.replace('"',' ')
        words = words.replace("'",' ')
        return words.strip()
    return words

    
def remove_grammar(review):
    sentences = nltk.sent_tokenize(review)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    result_review = []
    for sentence in sentences:
        if sentences.strip():
            tagged_review = nltk.tag._pos_tag(sentence, tagset, tagger)
            cp = nltk.RegexpParser(grammar)
            result = cp.parse(tagged_review)
            result_review.append(traverseTree(result))
    return ''.join([word for word in result_review])
    
# Remove email
def remove_email(words):
    mod_words = ''
    if words:
        if words.strip():
            for word in words.split(' '):
                if (word.strip().lower()=='email') or (word.strip().lower()=='phn') or (word.strip().lower()=='phone') or (len(word.strip())<=1):
                    continue
                elif not re.match(r"[^@]+@[^@]+\.[^@]+", word.lower()):
                    mod_words = mod_words+' '+word
                #else:   
    else:
        return words
    return mod_words.strip()

In [17]:
cleanse_text(hi)

'hi im tri my best'

In [18]:
import os

os.getcwd()

'/Users/admin/Desktop/tryhard/marketAnalysis/NLPFinalClust/Isita'

In [19]:
df1 = pd.read_csv('../RawClustersOldBios/woohoo1/3.csv')

In [23]:
df1['Bios']=df1['Bios'].apply(cleanse_text)

In [25]:
df1['Bios']

0     Internet nerd Award win coff trailblaz Social ...
1     Pop cultur trailblaz Coff maven Award win beer...
2     Problem solver Avid music buff Beer trailblaz ...
3     Evil social media lover Student Amateur thinke...
4     Friend problem solver Troublemak Subt charm so...
5     Beer geek Unapologet zombi ninja Award win mus...
6     Evil social media lover Student Amateur thinke...
7     Friend problem solver Troublemak Subt charm so...
8     Pop cultur geek Friend troublemak Student Weba...
9     Pop cultur trailblaz Coff maven Award win beer...
10    Avid coff geek Hipster friend twitter guru Fal...
11    Infuriat humbl zombi trailblaz Wannab tv ninja...
12    Pop cultur ninja Entrepreneur Profess writer C...
13    Avid coff geek Hipster friend twitter guru Fal...
14    Tv scholar Gener social media junki Extreme al...
15    Prone to fit of apathi Fall down lot Bacon gee...
16    Friend problem solver Troublemak Subt charm so...
17    Bacon advoc Troublemak Avid social media e

In [52]:
vectorizer = CountVectorizer(analyzer='word',stop_words='english',decode_error='ignore',binary=True)    
    
counts = vectorizer.fit_transform(df1['Bios'])

In [54]:
from sklearn.metrics import silhouette_samples,silhouette_score, davies_bouldin_score
from sklearn.cluster import AgglomerativeClustering
n = 30

for cluster_num in range(2,n):
#     kmeans = KMeans(n_clusters=10,n_jobs=-1)
    
    hac = AgglomerativeClustering(n_clusters = cluster_num)
    labels = hac.fit_predict(counts.toarray())
    sil = silhouette_score(counts.toarray(), labels)
    db = davies_bouldin_score(counts.toarray(), labels)
    print('clus {}: {}, {}'.format(cluster_num, sil, db))

clus 2: 0.12207304161333878, 0.9169925009260752
clus 3: 0.14429030773375967, 1.5245901847120844
clus 4: 0.17274515923500525, 2.1087251289715496
clus 5: 0.2127270774327061, 2.0606431798697527
clus 6: 0.24732877171611725, 1.8692173412589843
clus 7: 0.2976040439340036, 1.6444848045985803
clus 8: 0.3434042424101895, 1.509522569324341
clus 9: 0.40164208534275736, 1.3334035594742384
clus 10: 0.46090134460201665, 1.1863830843825078
clus 11: 0.5096317295574622, 1.1167421529952601
clus 12: 0.5506100811091011, 1.09399865733298
clus 13: 0.5793906360264066, 1.1746593959851357
clus 14: 0.6066628043517414, 1.141135341777687
clus 15: 0.6117350543368423, 1.1308779203627994
clus 16: 0.6159718329068953, 1.0533045642716796
clus 17: 0.6671871205232816, 0.9937706356137612
clus 18: 0.6949958147929114, 0.9717567761955207
clus 19: 0.6992387466269874, 0.9120895362152738
clus 20: 0.7017572914435898, 0.8189064642990621
clus 21: 0.7052216161067173, 0.7582423029123139
clus 22: 0.7075676785289413, 0.713517271308251

In [55]:
for cluster_num in range(2,n):
    kmeans = KMeans(n_clusters=10,n_jobs=-1)
    
#     hac = AgglomerativeClustering(n_clusters = cluster_num)
    labels = kmeans.fit_predict(counts.toarray())
    sil = silhouette_score(counts.toarray(), labels)
    db = davies_bouldin_score(counts.toarray(), labels)
    print('clus {}: {}, {}'.format(cluster_num, sil, db))

/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/

clus 2: 0.39179567263128856, 1.3261213311980247
clus 3: 0.40121610448856865, 1.3148985553140984
clus 4: 0.39179567263128856, 1.3261213311980247
clus 5: 0.3850869430594562, 1.3063133009730645
clus 6: 0.39695290550949225, 1.2590366117358736
clus 7: 0.40567471546457656, 1.3421103275899737
clus 8: 0.39179567263128856, 1.3261213311980247


/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/

clus 9: 0.3861824699270198, 1.3338478691944995
clus 10: 0.40566238819817146, 1.3396382326099312
clus 11: 0.39589618404768195, 1.3693736994401933
clus 12: 0.38113503953180244, 1.4456395066934062
clus 13: 0.39296693220223955, 1.3490227732293085
clus 14: 0.39945161506806487, 1.3473404570735017
clus 15: 0.3727972213933558, 1.3876040205203728


/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/

clus 16: 0.3850869430594562, 1.3063133009730643
clus 17: 0.3853260729541909, 1.3079160857800232
clus 18: 0.39307378134620025, 1.363110841505004
clus 19: 0.3912239781124687, 1.337365937065109
clus 20: 0.39714135654714394, 1.292654456522563
clus 21: 0.39927392523714755, 1.2905176247269874
clus 22: 0.3938320360138223, 1.3923581156328897


/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/cs185c/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
/opt/anaconda2/envs/

clus 23: 0.39813921795354495, 1.2691770770732924
clus 24: 0.40567471546457656, 1.3421103275899737
clus 25: 0.39662043828245885, 1.3646406691005986
clus 26: 0.41600167771744423, 1.256073172636458
clus 27: 0.38852664332330633, 1.4002283865106278
clus 28: 0.4060370101510312, 1.3148881760682216
clus 29: 0.417945290809076, 1.3305766388726812


In [60]:
n = 10
hac = AgglomerativeClustering(n_clusters = n)
labels1 = hac.fit_predict(counts.toarray())
df1['cluster_num'] = labels

In [34]:
os.mkdir('nlp_clus')

FileExistsError: [Errno 17] File exists: 'nlp_clus'

In [61]:
def please(df, labels, n, someWord):
    #df_ori = pd.read_csv(filepath)
    df['cluster_num'] = labels
    #os.mkdir(os.getcwd() + '/' + someWord)
    
    
    for cluster in range(n):
        yes = df[df['cluster_num'] == cluster]
        filename = '{}/{}.csv'.format(someWord, str(cluster))
        yes.to_csv(filename)
    return df


In [62]:
please(df1,labels1,n,'nlp_clus')

,Unnamed: 0,Unnamed: 0.1,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #,cluster_num
0,4,4,Internet nerd Award win coff trailblaz Social ...,3,2,1,1,7,1,2,1,8,2,1,3,2
1,10,10,Pop cultur trailblaz Coff maven Award win beer...,1,7,2,1,10,7,2,2,8,1,1,3,1
2,22,22,Problem solver Avid music buff Beer trailblaz ...,3,5,2,1,5,7,2,3,5,3,1,3,0
3,43,43,Evil social media lover Student Amateur thinke...,4,1,2,1,4,3,2,2,9,3,1,3,7
4,50,50,Friend problem solver Troublemak Subt charm so...,1,5,2,1,5,7,2,3,9,1,1,3,0
5,75,75,Beer geek Unapologet zombi ninja Award win mus...,3,4,2,1,6,10,2,2,4,3,1,3,2
6,88,88,Evil social media lover Student Amateur thinke...,2,2,3,1,9,1,2,1,9,1,1,3,7
7,95,95,Friend problem solver Troublemak Subt charm so...,3,7,3,1,4,3,2,2,4,1,1,3,0
8,113,113,Pop cultur geek Friend troublemak Student Weba...,3,2,1,1,4,9,2,2,3,3,1,3,2
9,115,115,Pop cultur trailblaz Coff maven Award win beer...,4,7,1,1,4,5,2,3,4,2,1,3,1


In [64]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words='english',decode_error='ignore',binary=True)    
    
tf_counts = vectorizer.fit_transform(df1['Bios'])

n = 30

for cluster_num in range(2,n):
#     kmeans = KMeans(n_clusters=10,n_jobs=-1)
    
    hac = AgglomerativeClustering(n_clusters = cluster_num)
    labels = hac.fit_predict(counts.toarray())
    sil = silhouette_score(counts.toarray(), labels)
    db = davies_bouldin_score(counts.toarray(), labels)
    print('clus {}: {}, {}'.format(cluster_num, sil, db))

clus 2: 0.12207304161333878, 0.9169925009260752
clus 3: 0.14429030773375967, 1.5245901847120844
clus 4: 0.17274515923500525, 2.1087251289715496
clus 5: 0.2127270774327061, 2.0606431798697527
clus 6: 0.24732877171611725, 1.8692173412589843
clus 7: 0.2976040439340036, 1.6444848045985803
clus 8: 0.3434042424101895, 1.509522569324341
clus 9: 0.40164208534275736, 1.3334035594742384
clus 10: 0.46090134460201665, 1.1863830843825078
clus 11: 0.5096317295574622, 1.1167421529952601
clus 12: 0.5506100811091011, 1.09399865733298
clus 13: 0.5793906360264066, 1.1746593959851357
clus 14: 0.6066628043517414, 1.141135341777687
clus 15: 0.6117350543368423, 1.1308779203627994
clus 16: 0.6159718329068953, 1.0533045642716796
clus 17: 0.6671871205232816, 0.9937706356137612
clus 18: 0.6949958147929114, 0.9717567761955207
clus 19: 0.6992387466269874, 0.9120895362152738
clus 20: 0.7017572914435898, 0.8189064642990621
clus 21: 0.7052216161067173, 0.7582423029123139
clus 22: 0.7075676785289413, 0.713517271308251

In [65]:
n = 13
hac = AgglomerativeClustering(n_clusters = n)
labels1 = hac.fit_predict(counts.toarray())
df1['cluster_num'] = labels

os.mkdir('nlp_clus_tfidf')
please(df1,labels1,n,'nlp_clus_tfidf')

,Unnamed: 0,Unnamed: 0.1,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,cluster #,cluster_num
0,4,4,Internet nerd Award win coff trailblaz Social ...,3,2,1,1,7,1,2,1,8,2,1,3,1
1,10,10,Pop cultur trailblaz Coff maven Award win beer...,1,7,2,1,10,7,2,2,8,1,1,3,0
2,22,22,Problem solver Avid music buff Beer trailblaz ...,3,5,2,1,5,7,2,3,5,3,1,3,5
3,43,43,Evil social media lover Student Amateur thinke...,4,1,2,1,4,3,2,2,9,3,1,3,7
4,50,50,Friend problem solver Troublemak Subt charm so...,1,5,2,1,5,7,2,3,9,1,1,3,10
5,75,75,Beer geek Unapologet zombi ninja Award win mus...,3,4,2,1,6,10,2,2,4,3,1,3,1
6,88,88,Evil social media lover Student Amateur thinke...,2,2,3,1,9,1,2,1,9,1,1,3,7
7,95,95,Friend problem solver Troublemak Subt charm so...,3,7,3,1,4,3,2,2,4,1,1,3,10
8,113,113,Pop cultur geek Friend troublemak Student Weba...,3,2,1,1,4,9,2,2,3,3,1,3,1
9,115,115,Pop cultur trailblaz Coff maven Award win beer...,4,7,1,1,4,5,2,3,4,2,1,3,0
